In [ ]:
from genscai import paths
import chromadb

DB_PATH = str(paths.output / "medrxiv.db")

client = chromadb.PersistentClient(path=DB_PATH)
client.list_collections()

In [ ]:
from genscai.tools import MCPClient

import nest_asyncio
nest_asyncio.apply()

mcp_config = {
    "mcpServers": {
        "genscai": {"command": "python", "args": ["../genscai/tools.py"]}
    }
}

# mcp_client = MCPClient(mcp_config)
mcp_client = MCPClient()

tools = mcp_client.list_tools()
for tool in tools:
    print(f"Tool Name: {tool.name}")    
    print(f"Description: {tool.description}")
    print(f"Input schema: {tool.inputSchema}")
    print("-" * 40)

In [ ]:
tool_results = mcp_client.call_tool(
    tool_name="search_research_articles",
    params={"search_request": "malaria modeling"}
)

for result in tool_results:
    print(f"Result type: {result.type}")
    print(f"Result text:\n{result.text}")

In [ ]:
from genscai.models import OllamaClient

model_client = OllamaClient(OllamaClient.MODEL_MISTRAL_SMALL_3_1_24B)

response = model_client.chat_streamed(
    {
        "role": "system",
        "content": "You are a helpful assistant that can use tools to provide brief answers to questons about infectious diseases and disease modeling. Please briefly introduce yourself."
    }
)

for chunk in response:
    print(chunk, end="", flush=True)

In [ ]:
response = model_client.chat_streamed(
    {
        "role": "user",
        "content": "current malaria modeling tools"
    },
    tools=mcp_client.get_tools()
)

for chunk in response:
    print(chunk, end="", flush=True)

In [ ]:
model_client.messages

In [ ]:
del model_client